In [2]:
%load_ext autoreload
%autoreload 1
%aimport my, model, check_budget

import sys
import numpy as np
import pandas as pd

import os
import gc
import matplotlib.pyplot as plt
import importlib
import pickle

import my
from my import p
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 200)
pd.set_option("max_colwidth", 45)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from sklearn.model_selection import train_test_split

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)
     
out = "out/"
da = 'data/'

SEED = 34
np.random.seed(SEED)
N_CPU = os.cpu_count()
os.makedirs(out,exist_ok=True)

In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn

import model as m
from check_budget import check_budget1 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
bins_path = da+"nn_bins.pickle"
model_path = da+"nn_weights.ckpt"
qu_path = da+"quantiles.json"

bins = my.load_pickle(bins_path)
features = bins.pop("features")

import json
with open(qu_path, "rb") as f:
    qs = json.load(f)

In [6]:
df = my.read_trans(da+'transactions_finetune.csv', sort=False, add_pos=False)
df = my.digitize_cat(df, bins)
df = df.groupby(["user_id"])[features].apply(lambda x: x.values.transpose()[:, -300:].tolist()).apply(lambda x: np.array(x,dtype=np.int32)).reset_index().rename(columns={0: "sequences"})
p(df.shape)
df[:3]

(7080, 2)


,user_id,sequences
0,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1..."
1,140,"[[13, 14, 14, 15, 15, 6, 15, 17, 14, 14, ..."
2,196,"[[16, 3, 3, 8, 13, 15, 19, 5, 8, 14, 15, ..."


In [7]:
df_atk = my.read_trans(out+'atk_train_26_2.csv', sort=False, add_pos=False)
df_atk = my.digitize_cat(df_atk, bins)
df_atk = df_atk.groupby(["user_id"])[features].apply(lambda x: x.values.transpose()[:, -300:].tolist()).apply(lambda x: np.array(x,dtype=np.int32)).reset_index().rename(columns={0: "sequences"})
p(df_atk.shape)
df_atk[:3]

(7080, 2)


,user_id,sequences
0,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1..."
1,140,"[[13, 14, 14, 15, 15, 6, 15, 17, 14, 14, ..."
2,196,"[[16, 3, 3, 8, 13, 15, 19, 5, 8, 14, 15, ..."


In [8]:
y_train = pd.read_csv(da+'target_finetune.csv').sort_values('user_id')
y_train[:3]

,user_id,target
5855,69,0
2515,140,0
1295,196,0


In [9]:
df['atk'] = 0
df_atk['atk'] = 1
df[:3]

,user_id,sequences,atk
0,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1...",0
1,140,"[[13, 14, 14, 15, 15, 6, 15, 17, 14, 14, ...",0
2,196,"[[16, 3, 3, 8, 13, 15, 19, 5, 8, 14, 15, ...",0


In [10]:
Xy = pd.concat([df,df_atk],ignore_index=True).merge(y_train,on='user_id').sort_values('user_id').reset_index(drop=True)
p(Xy.shape)
Xy[:3]

(14160, 4)


,user_id,sequences,atk,target
0,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1...",0,0
1,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1...",1,0
2,140,"[[13, 14, 14, 15, 15, 6, 15, 17, 14, 14, ...",0,0


In [11]:
model = m.get_model(model_path=da+'nn_weights.ckpt',eval_mode=True, seed=SEED, device=device)

fixed rnn
eval mode


In [12]:
preds1 = m.rel_pred(Xy, model, target_col='y_pred', N=30, seed=34, batch_size=1024)
preds1[:3]

,user_id,y_pred
0,69,0.008
1,69,0.027
2,140,0.133


In [13]:
from sklearn.metrics import roc_auc_score

def calc_auc(Xy, preds):
    q = pd.concat([Xy[['user_id','atk','target']],preds],axis=1)
    real_d = q[q.atk==0]
    real_auc = roc_auc_score(real_d['target'], real_d['y_pred'])
    
    atk_d = q[q.atk==1]
    atk_auc = roc_auc_score(atk_d['target'], atk_d['y_pred'])
    
    mean_auc = 2 / ((1/real_auc)  + (1/atk_auc))
    
    print(f'real_auc: {real_auc}, atk_auc: {atk_auc}, mean_auc: {mean_auc}')

In [14]:
calc_auc(Xy, preds1)

real_auc: 0.6924010085561569, atk_auc: 0.6257341926064592, mean_auc: 0.6573817096556462


In [15]:
Xy = my.add_folds(Xy,strat_col='target',group_col='user_id')
Xy[:3]

StratifiedGroupKFold


,user_id,sequences,atk,target,fold
0,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1...",0,0,0
1,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1...",1,0,0
2,140,"[[13, 14, 14, 15, 15, 6, 15, 17, 14, 14, ...",0,0,2


In [16]:
def calc_folds_auc(Xy, preds, folds = None):
    q = pd.concat([Xy,preds],axis=1)
    for fold in sorted(q['fold'].unique()):
        if folds and (fold not in (folds)):
            continue
        real_d = q[(q.atk==0) & (q.fold == fold)]
        real_auc = roc_auc_score(real_d['target'], real_d['y_pred'])

        atk_d = q[(q.atk==1) & (q.fold == fold)]
        atk_auc = roc_auc_score(atk_d['target'], atk_d['y_pred'])

        mean_auc = 2 / ((1/real_auc)  + (1/atk_auc))

        print(f'FOLD: {fold} real_auc: {real_auc}, atk_auc: {atk_auc}, mean_auc: {mean_auc}')
        
calc_folds_auc(Xy, preds1)

FOLD: 0 real_auc: 0.6885281951934656, atk_auc: 0.6310278025027489, mean_auc: 0.6585251929174158
FOLD: 1 real_auc: 0.6997497785306871, atk_auc: 0.6426650917737751, mean_auc: 0.6699937040120971
FOLD: 2 real_auc: 0.7285193289569657, atk_auc: 0.6581084366642354, mean_auc: 0.6915262027004891
FOLD: 3 real_auc: 0.6539406155253307, atk_auc: 0.5498568742695507, mean_auc: 0.5973990573313676
FOLD: 4 real_auc: 0.6938813025210084, atk_auc: 0.6339679621848741, mean_auc: 0.6625729697638036


In [17]:
# def rel_pred_1(sp, model, target_col='y_pred', N=30, size=300, replace=False, seed=34, batch_size=1024):
#     rng = np.random.default_rng(seed)
#     all_preds = []
#     for _ in range(N):
#         d_ = sp[['user_id','sequences']].copy()
#         d_['sequences'] = d_['sequences'].apply(lambda x:rng.choice(x,axis=1,size=size, replace=replace))

#         preds = m.one_pred(model, d_, batch_size=batch_size)
#         all_preds.append(preds['y_pred'].to_numpy())
#         # print(i, '|', auc(y_train, preds)) 
        
#     all_preds = np.mean(all_preds, axis=0)
#     preds.columns = ['user_id', target_col]
#     preds[target_col] = all_preds
#     return preds

In [19]:
# for s in [300, 290, 280, 250, 200]:
#     for rl in [False, True]:
#         preds2 = rel_pred_1(Xy, model, target_col='y_pred', N=30, size=s, replace=rl, seed=34, batch_size=1024)
#         print(f'size: {s} replace: {rl} | ', end='')
#         calc_auc(Xy, preds2)

In [22]:
Xy = my.add_folds(Xy,strat_col='target',group_col='user_id')
Xy[:3]

StratifiedGroupKFold


,user_id,sequences,atk,target,fold
0,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1...",0,0,0
1,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1...",1,0,0
2,140,"[[13, 14, 14, 15, 15, 6, 15, 17, 14, 14, ...",0,0,2


In [23]:
Xy.groupby('fold')['target'].mean()

fold
0   0.050
1   0.033
2   0.032
3   0.030
4   0.040
Name: target, dtype: float64

In [24]:
Xy.groupby('fold')['atk'].mean()

fold
0   0.500
1   0.500
2   0.500
3   0.500
4   0.500
Name: atk, dtype: float64

In [25]:
# torch.cuda.is_initialized()

In [26]:
Xy[:2]

,user_id,sequences,atk,target,fold
0,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1...",0,0,0
1,69,"[[2, 9, 11, 13, 19, 13, 13, 20, 10, 13, 1...",1,0,0


In [27]:
Xy['sequences'].iloc[0][:,:280].shape

(7, 280)

In [28]:
from torch.utils.data import DataLoader, Dataset, default_collate

class AdvTrainDs(Dataset):
    def __init__(self, df, size=300, seed=34):
        
        self.X = df['sequences'].to_numpy()
        self.y = df['target'].to_numpy()

        self.size = size
        self.rng = np.random.default_rng(seed)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x, y = self.X[idx].copy(), self.y[idx]
        self.rng.shuffle(x,axis=1)
        if self.size < 300:
            x = x[:, :self.size]
        return x, y

In [29]:
class AdvValDs(Dataset):
    def __init__(self, df, N_mean = 3, seed=34):

        self.rng = np.random.default_rng(seed)
        
        self.X = df['sequences'].repeat(N_mean)
        self.X = self.X.apply(lambda x:self.rng.permutation(x,axis=1)).to_numpy()

        self.y = np.repeat(df['target'].to_numpy(),N_mean)
        self.atk = np.repeat(df['atk'].to_numpy(),N_mean)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x, y, atk = self.X[idx].copy(), self.y[idx], self.atk[idx]
        return x, y, atk

In [30]:
# class ValDs_No_Shuffle(Dataset):
#     def __init__(self, df):
        
#         self.X = df['sequences'].to_numpy()

#         self.y = df['target'].to_numpy()
#         self.atk = df['atk'].to_numpy()
        
#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, idx):
#         x, y, atk = self.X[idx].copy(), self.y[idx], self.atk[idx]
#         return x, y, atk

In [31]:
import pytorch_lightning as pl
from sklearn.metrics import roc_auc_score

class DefenceRnn(pl.LightningModule):
    def __init__(self, rnn_units=128, top_classifier_units=64, lr=3e-4, out_features=2):
        super().__init__()
        self._transaction_cat_embeddings = nn.ModuleList(
            [
                self._create_embedding_projection(*m.embedding_projections[feature])
                for feature in m.embedding_projections.keys()
            ]
        )

        self._spatial_dropout = nn.Dropout2d(0.5)
        concat_embed = sum([m.embedding_projections[x][1] for x in m.embedding_projections.keys()])
        self._gru = nn.GRU(input_size=concat_embed, hidden_size=rnn_units, batch_first=True, bidirectional=True,)

        self._hidden_size = rnn_units

        pooling_result_dimension = self._hidden_size * 2
        self._top_classifier = nn.Sequential(
            nn.Linear(in_features=pooling_result_dimension * 3, out_features=top_classifier_units,),
            nn.ReLU(),
            nn.Linear(in_features=top_classifier_units, out_features=out_features),
        )

        self.loss = nn.functional.binary_cross_entropy
        self.learning_rate = lr

    def configure_optimizers(self):
        opt = torch.optim.AdamW(lr=self.learning_rate, params=self.parameters())
        return opt
#         scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'max', factor=0.5, patience=4,threshold=0.0001, min_lr=0.00001,verbose=True)
#         return {"optimizer": opt,
#         "lr_scheduler": {
#             "scheduler": scheduler,
#             "monitor": "mean_auc",
#             "frequency": 1
#         },}

    def forward(self, transactions_cat_features, *, return_embedding=False):
        batch_size = transactions_cat_features.shape[0]

        last_hidden, states = self.get_emb(transactions_cat_features)
        probs = self.classify_emb(batch_size, last_hidden, states)

        if return_embedding:
            return probs, states
        else:
            return probs

    def classify_emb(self, batch_size, last_hidden, states):
        rnn_max_pool = states.max(dim=1)[0]
        rnn_avg_pool = states.sum(dim=1) / states.shape[1]
        last_hidden = torch.reshape(last_hidden.permute(1, 2, 0), shape=(batch_size, self._hidden_size * 2))
        combined = torch.cat([rnn_max_pool, rnn_avg_pool, last_hidden], dim=-1)
        # drop = nn.functional.dropout(combined, p=0.5)
        drop = nn.functional.dropout(combined, p=0.5,training = self.training)
        logit = self._top_classifier(drop)
        probs = nn.functional.softmax(logit, dim=1)
        return probs

    def get_emb(self, transactions_cat_features):
        embeddings = [
            embedding(transactions_cat_features[:, i]) for i, embedding in enumerate(self._transaction_cat_embeddings)
        ]
        concated_embeddings = torch.cat(embeddings, dim=-1).permute(0, 2, 1).unsqueeze(3)
        dropout_embeddings = self._spatial_dropout(concated_embeddings)
        dropout_embeddings = dropout_embeddings.squeeze(3).permute(0, 2, 1)
        states, last_hidden = self._gru(dropout_embeddings)
        return last_hidden, states

    def training_step(self, train_batch, batch_idx):
        x, y_label = train_batch
        batch_size = len(y_label)
        y_pred = self.forward(x)

        y_true = torch.zeros(batch_size, 2)
        y_true[range(y_true.shape[0]), y_label.long()] = 1
        y_true_dev = y_true.to(self.device)

        batch_loss = self.loss(y_pred, y_true_dev)

        self.log("train_loss", batch_loss, on_step=True, on_epoch=False)

        return {
            "loss": batch_loss,
        }

    def validation_step(self, val_batch, batch_idx):
        x, y_label, atk = val_batch
        batch_size = len(y_label)
        y_hat = self.forward(x)

        y_true = torch.zeros(batch_size, 2)
        y_true[range(y_true.shape[0]), y_label.long()] = 1
        y_true_dev = y_true.to(self.device)

        val_loss = self.loss(y_hat, y_true_dev)

        self.log('val_loss', val_loss, on_step=False, on_epoch=True)

        return (
            y_hat.detach()[:,1].cpu().numpy(),
            y_label.cpu().numpy(),
            atk.cpu().numpy())

        # self.val_metric(y_hat.detach()[:,1], y_label)

        # self.log('val_roc_auc',self.val_metric, on_step=False, on_epoch=True, prog_bar=True)

    def validation_epoch_end(self, outputs):
        y_hats = np.concatenate([o[0] for o in outputs])
        ys = np.concatenate([o[1] for o in outputs])
        atks = np.concatenate([o[2] for o in outputs]).astype(bool)

        fr = np.logical_not(atks)
        try:
            real_auc = roc_auc_score(ys[fr], y_hats[fr])
            atk_auc = roc_auc_score(ys[atks], y_hats[atks])

            mean_auc = 2 / ((1/real_auc)  + (1/atk_auc))
        except:
            print('no auc')
            real_auc,atk_auc,mean_auc = 0.,0.,0.

        print(f'real_auc: {real_auc}, atk_auc: {atk_auc}, mean_auc: {mean_auc}')

        self.log('mean_auc', mean_auc, on_step=False, on_epoch=True, prog_bar=True)

    @classmethod
    def _create_embedding_projection(cls, cardinality, embed_size, add_missing=True, padding_idx=0):
        add_missing = 1 if add_missing else 0
        return nn.Embedding(
            num_embeddings=cardinality + add_missing, embedding_dim=embed_size, padding_idx=padding_idx,
        )

In [32]:
pl.__version__

'1.9.3'

In [33]:
import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*exists and is not empty.*")
CHECKPOINT_DIR = 'ckpts'
os.makedirs(CHECKPOINT_DIR,exist_ok=True)

In [34]:
# torch.multiprocessing.set_start_method('spawn')

In [35]:
def defence_model(model_path='data/nn_weights.ckpt', is_pl=False, eval_mode=True, seed=20230206, device=None, lr=0.0001):
    pl.seed_everything(seed)
    if is_pl:
        model = DefenceRnn.load_from_checkpoint(model_path)
    else:
        model = DefenceRnn(lr=lr)
        model.load_state_dict(torch.load(model_path))
        
    model.to(device)
    if eval_mode:
        print('eval mode')
        model.eval()
    else:
        print('train mode')
    return model

In [37]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, TQDMProgressBar, ModelCheckpoint

tq = TQDMProgressBar(refresh_rate=5)

def train_fold(Xy, fold, repeat):
    FOLD_SEED = SEED + 10*fold + 100*repeat + 1000
    pl.seed_everything(FOLD_SEED)
    print(f'FOLD: {fold}, REPEAT: {repeat}, SEED: {FOLD_SEED}')
    
    Xy_train = Xy[Xy['fold']!=fold]
    Xy_val = Xy[Xy['fold']==fold]

    train_ds = AdvTrainDs(Xy_train, size = 250, seed=FOLD_SEED)
    val_ds = AdvValDs(Xy_val, seed=FOLD_SEED+10, N_mean=10)

    train_dl = m.my_get_dataloader(train_ds, device, batch_size=16, shuffle=True)
    val_dl = m.my_get_dataloader(val_ds, device, batch_size=1024, shuffle=False)

    model = defence_model(model_path=da+'nn_weights.ckpt',eval_mode=False, seed=FOLD_SEED, device=device, lr= 0.00003)
    
    es = EarlyStopping('mean_auc',patience=7,verbose=True,mode='max')

    chpt = ModelCheckpoint(dirpath=CHECKPOINT_DIR, filename=f'best_fold_{repeat}_{fold}',  monitor="mean_auc",mode='max')
    
    trainer = pl.Trainer(
    callbacks=[tq,es,chpt],
    max_epochs=500,
#     deterministic = True,
    accelerator='auto',
#     logger = False,
    log_every_n_steps = 50,
#     enable_model_summary = True if fold==0 else False,
    # val_check_interval = 1.,
#         auto_lr_find=True
    )
    trainer.fit(model, train_dl, val_dl)
    print(f'FOLD: {fold},REPEAT: {repeat}, BEST_RES: ',chpt.best_model_score.cpu().item())

In [38]:
# !rm -rf '/kaggle/working/ckpts'

In [42]:
# 250  lr = 0.00003
for repeat in range(3):
    Xy = my.add_folds(Xy,strat_col='target',group_col='user_id', n_folds=5, random_state=SEED+1000+100*repeat)
    for fold in range(5):
        
        train_fold(Xy, fold, repeat)
        model_tr = defence_model(model_path=f'ckpts/best_fold_{repeat}_{fold}.ckpt',is_pl=True, eval_mode=True, seed=SEED, device=device)
        preds1 = m.rel_pred(Xy, model_tr, target_col='y_pred', N=30, seed=SEED, batch_size=2048)
        calc_folds_auc(Xy, preds1)
        del model_tr, preds1
        torch.cuda.empty_cache()
        gc.collect()
    #     break

StratifiedGroupKFold
FOLD: 0, REPEAT: 0, SEED: 1034
train mode


Sanity Checking: 0it [00:00, ?it/s]

real_auc: 0.6138917004048583, atk_auc: 0.6289285714285714, mean_auc: 0.6213191704346946


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

real_auc: 0.6708652067824006, atk_auc: 0.6350297623673127, mean_auc: 0.6524557991381392


Validation: 0it [00:00, ?it/s]

real_auc: 0.6698386770899709, atk_auc: 0.6359970781592403, mean_auc: 0.6524793638936158


Validation: 0it [00:00, ?it/s]

real_auc: 0.6673513824347637, atk_auc: 0.6315967548917519, mean_auc: 0.6489819807368797
FOLD: 4,REPEAT: 2, BEST_RES:  0.6577085994361204
eval mode
FOLD: 0 real_auc: 0.7739925088976267, atk_auc: 0.7592589714498857, mean_auc: 0.7665549503755703
FOLD: 1 real_auc: 0.7098184073990527, atk_auc: 0.7219292803970223, mean_auc: 0.7158226221478565
FOLD: 2 real_auc: 0.7523487887867553, atk_auc: 0.7664669138184957, mean_auc: 0.7593422339093928
FOLD: 3 real_auc: 0.8112335584784928, atk_auc: 0.8029175765578184, mean_auc: 0.8070541458700199
FOLD: 4 real_auc: 0.6769656372876615, atk_auc: 0.6436286775562221, mean_auc: 0.6598763798706364


In [43]:
!ls -la ckpts

total 70748
drwxr-xr-x 2 root root    4096 Mar 31 18:36 .
drwxr-xr-x 5 root root    4096 Mar 31 16:53 ..
-rw-r--r-- 1 root root 4827216 Mar 31 16:59 best_fold_0_0.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 17:04 best_fold_0_1.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 17:07 best_fold_0_2.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 17:13 best_fold_0_3.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 17:21 best_fold_0_4.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 17:25 best_fold_1_0.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 17:37 best_fold_1_1.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 17:50 best_fold_1_2.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 17:58 best_fold_1_3.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 18:05 best_fold_1_4.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 18:08 best_fold_2_0.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 18:13 best_fold_2_1.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 18:23 best_fold_2_2.ckpt
-rw-r--r-- 1 root root 4827216 Mar 31 18:33 best_fold_2_3.ckpt
-rw-r--r-- 1 